# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 31 2022 12:35PM,238437,19,ADV80002938,"AdvaGen Pharma, Ltd",Released
1,May 31 2022 12:35PM,238437,19,ADV80002939,"AdvaGen Pharma, Ltd",Released
2,May 31 2022 12:35PM,238437,19,ADV80002940,"AdvaGen Pharma, Ltd",Released
3,May 31 2022 12:35PM,238435,15,8086738,"Carwin Pharmaceutical Associates, LLC",Released
4,May 31 2022 12:34PM,238434,10,MET210738,"Methapharm, Inc.",Released
5,May 31 2022 12:34PM,238434,10,MET210744,"Methapharm, Inc.",Released
6,May 31 2022 12:34PM,238434,10,MET210745,"Methapharm, Inc.",Released
7,May 31 2022 12:30PM,238432,19,DEX0006881,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
8,May 31 2022 12:30PM,238432,19,DEX0006882,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
9,May 31 2022 12:30PM,238432,19,DEX0006883,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
53,238430,Released,2
54,238432,Released,3
55,238434,Released,3
56,238435,Released,1
57,238437,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238430,NaN,NaN,2.0
238432,NaN,NaN,3.0
238434,NaN,NaN,3.0
238435,NaN,NaN,1.0
238437,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238317,0.0,9.0,1.0
238319,0.0,0.0,1.0
238334,1.0,7.0,6.0
238341,0.0,0.0,35.0
238346,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238317,0,9,1
238319,0,0,1
238334,1,7,6
238341,0,0,35
238346,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238317,0,9,1
1,238319,0,0,1
2,238334,1,7,6
3,238341,0,0,35
4,238346,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238317,,9,1
1,238319,,,1
2,238334,1,7,6
3,238341,,,35
4,238346,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 31 2022 12:35PM,238437,19,"AdvaGen Pharma, Ltd"
3,May 31 2022 12:35PM,238435,15,"Carwin Pharmaceutical Associates, LLC"
4,May 31 2022 12:34PM,238434,10,"Methapharm, Inc."
7,May 31 2022 12:30PM,238432,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
10,May 31 2022 12:26PM,238430,16,Sartorius Bioprocess Solutions
12,May 31 2022 12:03PM,238429,102,"Senseonics, Incorporated"
13,May 31 2022 12:00PM,238428,10,Twinlab Consolidated Corporation
14,May 31 2022 11:57AM,238427,19,"AdvaGen Pharma, Ltd"
24,May 31 2022 11:49AM,238426,18,Innova Softgel LLC
25,May 31 2022 11:38AM,238424,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 31 2022 12:35PM,238437,19,"AdvaGen Pharma, Ltd",,,3
1,May 31 2022 12:35PM,238435,15,"Carwin Pharmaceutical Associates, LLC",,,1
2,May 31 2022 12:34PM,238434,10,"Methapharm, Inc.",,,3
3,May 31 2022 12:30PM,238432,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,3
4,May 31 2022 12:26PM,238430,16,Sartorius Bioprocess Solutions,,,2
5,May 31 2022 12:03PM,238429,102,"Senseonics, Incorporated",,,1
6,May 31 2022 12:00PM,238428,10,Twinlab Consolidated Corporation,,,1
7,May 31 2022 11:57AM,238427,19,"AdvaGen Pharma, Ltd",,7,3
8,May 31 2022 11:49AM,238426,18,Innova Softgel LLC,,,1
9,May 31 2022 11:38AM,238424,10,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 12:35PM,238437,19,"AdvaGen Pharma, Ltd",3,,
1,May 31 2022 12:35PM,238435,15,"Carwin Pharmaceutical Associates, LLC",1,,
2,May 31 2022 12:34PM,238434,10,"Methapharm, Inc.",3,,
3,May 31 2022 12:30PM,238432,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3,,
4,May 31 2022 12:26PM,238430,16,Sartorius Bioprocess Solutions,2,,
5,May 31 2022 12:03PM,238429,102,"Senseonics, Incorporated",1,,
6,May 31 2022 12:00PM,238428,10,Twinlab Consolidated Corporation,1,,
7,May 31 2022 11:57AM,238427,19,"AdvaGen Pharma, Ltd",3,7,
8,May 31 2022 11:49AM,238426,18,Innova Softgel LLC,1,,
9,May 31 2022 11:38AM,238424,10,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 12:35PM,238437,19,"AdvaGen Pharma, Ltd",3,,
1,May 31 2022 12:35PM,238435,15,"Carwin Pharmaceutical Associates, LLC",1,,
2,May 31 2022 12:34PM,238434,10,"Methapharm, Inc.",3,,
3,May 31 2022 12:30PM,238432,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3,,
4,May 31 2022 12:26PM,238430,16,Sartorius Bioprocess Solutions,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 12:35PM,238437,19,"AdvaGen Pharma, Ltd",3.0,NaN,NaN
1,May 31 2022 12:35PM,238435,15,"Carwin Pharmaceutical Associates, LLC",1.0,NaN,NaN
2,May 31 2022 12:34PM,238434,10,"Methapharm, Inc.",3.0,NaN,NaN
3,May 31 2022 12:30PM,238432,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3.0,NaN,NaN
4,May 31 2022 12:26PM,238430,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 12:35PM,238437,19,"AdvaGen Pharma, Ltd",3.0,0.0,0.0
1,May 31 2022 12:35PM,238435,15,"Carwin Pharmaceutical Associates, LLC",1.0,0.0,0.0
2,May 31 2022 12:34PM,238434,10,"Methapharm, Inc.",3.0,0.0,0.0
3,May 31 2022 12:30PM,238432,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,3.0,0.0,0.0
4,May 31 2022 12:26PM,238430,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5721491,460.0,61.0,12.0
102,476849,2.0,0.0,0.0
12,953397,76.0,1.0,0.0
15,715071,3.0,9.0,0.0
16,238430,2.0,0.0,0.0
18,1191971,5.0,0.0,0.0
19,953650,12.0,9.0,0.0
20,1191942,51.0,8.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5721491,460.0,61.0,12.0
1,102,476849,2.0,0.0,0.0
2,12,953397,76.0,1.0,0.0
3,15,715071,3.0,9.0,0.0
4,16,238430,2.0,0.0,0.0
5,18,1191971,5.0,0.0,0.0
6,19,953650,12.0,9.0,0.0
7,20,1191942,51.0,8.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,460.0,61.0,12.0
1,102,2.0,0.0,0.0
2,12,76.0,1.0,0.0
3,15,3.0,9.0,0.0
4,16,2.0,0.0,0.0
5,18,5.0,0.0,0.0
6,19,12.0,9.0,0.0
7,20,51.0,8.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,460.0
1,102,Released,2.0
2,12,Released,76.0
3,15,Released,3.0
4,16,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20
Status,,,,,,,,
Completed,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Executing,61.0,0.0,1.0,9.0,0.0,0.0,9.0,8.0
Released,460.0,2.0,76.0,3.0,2.0,5.0,12.0,51.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20
0,Completed,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Executing,61.0,0.0,1.0,9.0,0.0,0.0,9.0,8.0
2,Released,460.0,2.0,76.0,3.0,2.0,5.0,12.0,51.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20
0,Completed,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Executing,61.0,0.0,1.0,9.0,0.0,0.0,9.0,8.0
2,Released,460.0,2.0,76.0,3.0,2.0,5.0,12.0,51.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()